## PHE Charts

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Common Libraries

Import libraries for working with PHE data

In [1]:
import os
from datetime import date, datetime, timedelta

import csv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

import common_core
import phe_core
import ons_population

## Area Class

Supports a single area - nation, region, ltla, etc

- Load data into ndarray from CSV
- Plot data

In [2]:
CORONAVIRUS_URL = "coronavirus.data.gov.uk"

GITHUB_URL = "https://logiqx.github.io/covid-stats/daily-trends"

# Cases:      Apply 3 day "lag" to the PHE data (national / regional) + 3 days relating to MA calculation
# Admissions: Apply 0 day "lag" to the PHE data (national / regional) + 3 days relating to MA calculation
# Deaths:     Apply 5 day "lag" to the PHE data (national / regional) + 3 days relating to MA calculation

categoryLags = \
{
    "cases": 3,
    "admissions": 0,
    "deaths": 5
}

regionColors = \
{
    "North East": "navy",
    "North East and Yorkshire": "navy",
    "Yorkshire and The Humber": "cornflowerblue",
    "Yorkshire": "cornflowerblue",
    "North West": "deepskyblue",
    "East Midlands": "lawngreen",
    "West Midlands": "green",
    "Midlands": "lawngreen",
    "East of England": "darkorange",
    "London": "red",
    "South East": "gold",
    "South West": "darkgrey",
    "England": "red",
    "Wales": "gold",
    "Northern Ireland": "green",
    "Scotland": "blue"
}

In [3]:
class Area(common_core.Printable):

    def __init__(self, areaType, areaName, areaCode=None):
        """Initialisise the area object"""

        self.data = {}

        self.areaType = areaType
        self.areaName = areaName
        self.areaCode = areaCode

        self.safeName = common_core.getSafeName(areaName)
        self.csvName = self.safeName + '.csv'
        

    def load(self, period, dataType):
        """Load demographic data - cases or deaths"""

        fileName = os.path.join(common_core.dataDir, phe_core.PHE_DASHBOARD, "csv", period, dataType, self.areaType, self.csvName)
        partName = common_core.getPartName(fileName)

        try:
            with open(fileName, 'r') as f:
                reader = csv.reader(f, delimiter = ',')

                dtype = []
                converters = {}
                colNames = next(reader)

                for i in range(len(colNames)):
                    colName = colNames[i]
                    if colName == "date":
                        dtype.append((colName, "U10"))
                    elif colName == "areaName":
                        dtype.append((colName, "U40"))
                    elif "RollingRate" in colName:
                        dtype.append((colName, "f8"))
                        converters[i] = lambda s: float(s or 0)
                    else:
                        dtype.append((colName, "u4"))
                        converters[i] = lambda s: int(s or 0)

                data = np.genfromtxt(f, dtype=dtype, converters=converters, delimiter=",")

            # Ensure period is present in data arrays
            if period not in self.data:
                self.data[period] = {}

            # Store the data
            self.data[period][dataType] = data

        # General catch all
        except:
            print(f"Failed to load {dataType} for {self.areaName}")
            raise


    def loadDaily(self):
        """Load PHE daily data from CSV files into ndarrays"""

        period = "daily"
        
        if self.areaType == "nation":
            self.load(period, "cases")
            self.load(period, "patients")
            self.load(period, "deaths")

        elif self.areaType == "nhsregion":
            self.load(period, "patients")

        else:
            self.load(period, "cases")
            self.load(period, "deaths")

## Areas Class

Combines multiple areas - region, etc

In [4]:
class Areas(common_core.Printable):

    def __init__(self, areaType):
        """Initialisise the areas object"""

        self.areaType = areaType
        self.areas = {}
        
        if self.areaType == "region":
            self.widerArea = common_core.ENGLAND
        elif self.areaType == "nation":
            self.widerArea = common_core.UNITED_KINGDOM

    
    def addArea(self, areaName, areaCode=None):
        """Add a new area which can then be loadeded from disk"""
        
        area = Area(self.areaType, areaName, areaCode=areaCode)

        self.areas[areaName] = area
        

    def loadArea(self, areaName):
        """Load PHE data for a single area"""
        
        self.areas[areaName].loadDaily()
        

    def prepCharts(self, startDate, category, lag, abbreviateNames):
        """Prepare data for matplotlib"""

        chart1 = f'Daily {category}'
        chart2 = f'Daily {category} per 100,000'
        chart3 = f'Daily {category} as % of 2021 max'
        
        charts = {
            chart1: {},
            chart2: {},
            chart3: {}
        }
        dates = []

        if self.areaType == "region" and category == "admissions":
            areasObj = nhsRegions
        else:
            areasObj = self

        for areaName in areasObj.areas:
            area = areasObj.areas[areaName]
            
            if abbreviateNames and " and " in areaName:
                seriesName = areaName[:areaName.find(" and ")]
            else:
                seriesName = areaName

            if category == "admissions":
                data = area.data[period]["patients"]
            else:
                data = area.data[period][category]

            minIdx = np.where(data["date"] == startDate)[0][-1]
            janIdx = np.where(data["date"] == "2021-01-01")[0][-1]

            counts = data[category]
            nonZeroIdx = np.nonzero(counts)

            numPersons = 0
            if self.areaType == "region" and category == "admissions":
                for nhsRegionMapping in common_core.nhsRegionMappings:
                    if common_core.nhsRegionMappings[nhsRegionMapping] == areaName:
                        for regionCode in common_core.regions:
                            if common_core.regions[regionCode] == nhsRegionMapping:
                                numPersons += population.getPopulation(regionCode)
            else:
                if self.areaType == "region":
                    for regionCode in common_core.regions:
                        if common_core.regions[regionCode] == areaName:
                            numPersons += population.getPopulation(regionCode)
                elif self.areaType == "nation":
                    for nationCode in common_core.nations:
                        if common_core.nations[nationCode] == areaName:
                            numPersons += population.getPopulation(nationCode)
                            
            if numPersons == 0:
                raise RuntimeError(f"Population unknown for {areaName}")
                
            # Calculate moving average
            counts = common_core.movingAverage(counts)
            
            # This is the daily rate but can multiply by 7 to get the weekly rate
            rates = 100000 * counts / numPersons

            # Calculate % of maximum since Jan 2021
            percentages = counts / np.max(counts[janIdx:]) * 100

            # It is possible that there were no significant values for this age band
            if len(nonZeroIdx[0]) > 0:
                
                # Ignore last X days of counts due to data lag and use of centred MA
                maxIdx = nonZeroIdx[0][-1] - lag - 3
                
                # First chart is just the centered moving average
                charts[chart1][seriesName] = counts[minIdx:maxIdx + 1]

                # Second chart is rates per 100,000
                charts[chart2][seriesName] = rates[minIdx:maxIdx + 1]

                # Third chart is the % of maximum since Jan 2021
                charts[chart3][seriesName] = percentages[minIdx:maxIdx + 1]

                # Ensure we have a full list of dates for the x-axis
                if maxIdx - minIdx + 1 > len(dates):
                    dates = []
                    for yyyymmdd in np.array(data["date"])[minIdx:maxIdx + 1]:
                        formattedDate = datetime.strptime(yyyymmdd, "%Y-%m-%d").strftime("%-d %b %y")
                        dates.append(formattedDate)

        return charts, dates


    def setLegend(self, axs, loc):
        """Set the legend"""
        
        for i in range(len(axs)):
            ax = axs[i]
            ax.legend(loc=loc, borderaxespad=1, fontsize="small")


    def setScale(self, axs, scale):
        """Set the scale - use log first to avoid warning when ymin equals 0"""

        for i in range(len(axs)):
            ax = axs[i]
            ax.set_yscale(scale)

            if scale == "linear":
                if i == 2:
                    # Format as percentage
                    ax.yaxis.set_major_formatter(tck.PercentFormatter())
                else:
                    # Ensure thousands are shown using commas and small numbers show decimals
                    ax.yaxis.set_major_formatter(tck.FuncFormatter(
                        lambda x, p: format(int(x), ',') if x.is_integer()
                        else format(x, ",.1f") if (x * 10).is_integer()
                        else format(x, ",.2f") if (x * 100).is_integer()
                        else format(x, ",.3f")))
                
                # Set ylim to zero when using linear scale
                ax.set_ylim(ymin=0)

            elif scale == "log":
                if i == 2:
                    # Format as percentage
                    ax.yaxis.set_major_formatter(tck.FuncFormatter(lambda x, p: format(int(x), ',') + "%"))

                    # Only show minor ticks labels if 10% is not visible
                    if ax.get_ylim()[0] > 10:
                        ax.yaxis.set_minor_formatter(tck.PercentFormatter())
                    else:
                        ax.yaxis.set_minor_formatter(tck.NullFormatter())

                else:
                    # Ensure thousands are shown using commas and small numbers show decimals
                    ax.yaxis.set_major_formatter(tck.FuncFormatter(
                        lambda x, p: format(int(x), ',') if x >= 1 or x == 0
                        else format(x, ",.4f") if x < .001
                        else format(x, ",.3f") if x < .01
                        else format(x, ",.2f") if x < .1
                        else format(x, ",.1f")))

                    # Do not show minor tick labels
                    ax.yaxis.set_minor_formatter(tck.NullFormatter())                   

            else:
                raise RuntimeError(f"Unsupported scale {scale}")


    def showRestrictions(self, ax, dates):
        """Show dates of restrictions"""

        # Tier 3 + Lockdown 2
        try:
            # Only show if 1 Sep is present
            x = dates.index("1 Sep 20")

            # Tier 3 #1
            try:
                x = dates.index("14 Oct 20")
                ax.axvline(x, color='darkgrey', linestyle=(0, (1, 3)), linewidth=1, label="Tier 3 for Liverpool (14/10)")
            except ValueError:
                pass

            # Tier 3 #2
            try:
                x = dates.index("23 Oct 20")
                ax.axvline(x, color='darkgrey', linestyle=(0, (1, 2)), linewidth=1, label="Tier 3 for NW/Y+H (23/10)")
            except ValueError:
                pass

            # Lockdown 2
            try:
                xmin = dates.index("5 Nov 20")
            except ValueError:
                xmin = -1

            try:
                xmax = dates.index("2 Dec 20")
            except ValueError:
                xmax = -1

            if xmin >= 0 or xmax >= 0:
                if xmin < 0:
                    xmin = 0
                if xmax < 0:
                    xmax = len(dates) - 1
                ax.axvspan(xmin, xmax, facecolor='lightgrey', alpha=0.25, label="Lockdown 2 (5/11 to 2/12)")

        except ValueError:
            pass
                
        # Tier 4 #1
        try:
            x = dates.index("20 Dec 20")
            ax.axvline(x, color='darkgrey', linestyle=(0, (4, 3, 1, 3, 1, 3)), linewidth=1, label="Tier 4 - London/E/SE (20/12)")
        except ValueError:
            pass

        # Tier 4 #2
        try:
            x = dates.index("26 Dec 20")
            ax.axvline(x, color='darkgrey', linestyle=(0, (4, 3, 1, 3)), linewidth=1, label="Tier 4 - rest of E/SE (26/12)")
        except ValueError:
            pass

        # Tier 4 #3
        try:
            x = dates.index("31 Dec 20")
            ax.axvline(x, color='darkgrey', linestyle=(0, (4, 3)), linewidth=1, label="Tier 4 - most areas (31/12)")
        except ValueError:
            pass

        # Lockdown 3
        try:
            x = dates.index("6 Jan 21")
            ax.axvline(x, color='darkgrey', linestyle=(0, (4, 1)), linewidth=1, label="Lockdown 3 (6/1)")
        except ValueError:
            pass

        # Step 1a
        try:
            x = dates.index("8 Mar 21")
            ax.axvline(x, color='darkgrey', linestyle=(0, (4, 2)), linewidth=1, label="Step 1a - Schools (8/3)")
        except ValueError:
            pass

        # Step 1b
        try:
            x = dates.index("29 Mar 21")
            ax.axvline(x, color='darkgrey', linestyle=(0, (4, 3)), linewidth=1, label="Step 1b - Outdoors (29/3)")
        except ValueError:
            pass

        # Step 2
        try:
            x = dates.index("12 Apr 21")
            ax.axvline(x, color='darkgrey', linestyle=(0, (4, 4)), linewidth=1, label="Step 2 - Hospitality (12/4)")
        except ValueError:
            pass

        # Step 3
        try:
            x = dates.index("17 May 21")
            ax.axvline(x, color='darkgrey', linestyle=(0, (4, 5)), linewidth=1, label="Step 3 - Indoors (17/5)")
        except ValueError:
            pass


    def plotCharts(self, axs, period, dataType, charts, dates, showRestrictions):
        """Plot data for daily cases / hospitalisations / deaths"""

        axTitles = [*charts.keys()]

        # Ignore the under 0-24 and 25-39 categories
        for i in range(len(axs)):
            
            # Determine the chart title and axis
            ax = axs[i]
            axTitle = axTitles[i]

            # Plot the data series
            plots = charts[axTitle]
            for plotName in plots:
                y_points = plots[plotName]
                x_points = np.arange(len(y_points))
                ax.plot(x_points, y_points, color=regionColors[plotName], label=plotName)

            # Add title, axis labels and legend
            ax.set_title(axTitle)
            
            # Change the x-axis to show the actual dates
            tickInterval = 7
            ax.set_xlim(xmin=-2, xmax=len(x_points) + 1)
            ax.set_xticks(np.arange(0, len(dates), tickInterval))
            ax.set_xticklabels(dates[::tickInterval], rotation=90)

            # Show restrictions such as tiers, etc
            if showRestrictions:
                self.showRestrictions(ax, dates)


    def plotCategory(self, startDate, period, axs, category, abbreviateNames=False, showRestrictions=False):
        """Plot data for a generic category"""

        lag = categoryLags[category]

        charts, dates = self.prepCharts(startDate, category, lag, abbreviateNames)

        self.plotCharts(axs, period, category, charts, dates, showRestrictions)
        

    def setHeader(self, fig, startDate):
        """Set the header / title"""

        formattedDate = datetime.strptime(startDate, "%Y-%m-%d").strftime("%-d %B %Y")
        fig.suptitle(f"COVID-19 in {self.widerArea} since {formattedDate}", y=0.96,
                     fontsize="x-large", fontweight="bold")
        
        textStr = 'All plots are shown as a 7 day centered moving average.'
        fig.text(0.5, 0.94, textStr, horizontalalignment='center', verticalalignment='top')

        lastRefresh = datetime.now().strftime("%a %-d %b %Y at %H:%M")
        textStr = f"Last refreshed on {lastRefresh}."
        fig.text(0.5, 0.92, textStr, horizontalalignment='center', verticalalignment='top')


    def setFooter(self, fig):
        """Set the footer - external links, etc."""

        textStr = f"This data was retrieved via the API at {CORONAVIRUS_URL} and plotted by @Mike_aka_Logiqx"
        fig.text(0.5, 0.05, textStr, horizontalalignment='center', verticalalignment='top')

        textStr = f"Further images plus the code and data can be found at {GITHUB_URL}"
        fig.text(0.5, 0.03, textStr, horizontalalignment='center', verticalalignment='top')


    def saveImage(self, fig, prefix, subDir=None):
        """Save the image and close the figure"""

        plt.show()

        baseName = prefix + ".png"

        partName = os.path.join("docs", "daily-trends", self.areaType + "s")
        if subDir:
            partName = os.path.join(partName, subDir)
        partName = os.path.join(partName, baseName)
        fileName = os.path.join(common_core.projdir, partName)

        dirName = os.path.dirname(fileName)
        if not os.path.exists(dirName):
            os.makedirs(dirName)

        print(f"Saving {partName}...")
        fig.savefig(fileName, bbox_inches='tight', facecolor='w')
        plt.close(fig)


    def createFigure(self, nrows, ncols, startDate, figsize=(16, 18), dpi=100):
        """Create a new figure with standard header and footer"""

        fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, dpi=dpi)
        
        plt.subplots_adjust(hspace=0.4)

        self.setHeader(fig, startDate)
        self.setFooter(fig)

        return fig, axs


    def plotSummary(self, period):
        """Plot data for cases, hospitalisations and deaths"""

        startDate = str(date.today() - timedelta(weeks=13))

        fig, axs = self.createFigure(3, 3, startDate)

        self.plotCategory(startDate, period, axs[0], "cases", abbreviateNames=True)
        self.plotCategory(startDate, period, axs[1], "admissions", abbreviateNames=True)
        self.plotCategory(startDate, period, axs[2], "deaths", abbreviateNames=True)

        for scale in ["log", "linear"]:
            for i in range(3):
                self.setScale(axs[i], scale)

                if scale == "linear":
                    self.setLegend(axs[i], "upper right")
                else:
                    self.setLegend(axs[i], "lower left")

            self.saveImage(fig, prefix="overview", subDir=scale)                   


    def plotDetail(self, period, category):
        """Plot data for cases, hospitalisations and deaths"""

        startDate = str(date.today() - timedelta(weeks=39))

        fig, axs = self.createFigure(3, 1, startDate, figsize=(12, 18))

        if self.areaType == "region":
            showRestrictions = True
        else:
            showRestrictions = False

        self.plotCategory(startDate, period, axs, category, showRestrictions=showRestrictions)

        for scale in ["log", "linear"]:
            self.setScale(axs, scale)

            self.setLegend(axs, "upper left")

            self.saveImage(fig, prefix=category, subDir=scale)    


    def plotCases(self, period):
        """Plot data for daily cases"""
        
        self.plotDetail(period, "cases")


    def plotAdmissions(self, period):
        """Plot data for daily admissions"""
        
        self.plotDetail(period, "admissions")


    def plotDeaths(self, period):
        """Plot data for daily deaths"""
        
        self.plotDetail(period, "deaths")
        

    def saveIndexPages(self):
        """Save HTML pages as index to images"""

        indexPageDetails = \
        {
            "TITLE_TEXT": "COVID-19",
            "H1_TEXT": "COVID-19",
            "H2_TEXT": f"Daily trends for {self.widerArea}",
            "CUSTOM_CSS": "../css/custom.css",
            "FURTHER_HREF": "../"
        }

        dirName = os.path.join(common_core.projdir, "docs", "daily-trends", self.areaType + "s")
        fileName = os.path.join(dirName, f"index.html")
        indexPage = common_core.IndexPage(fileName, indexPageDetails)

        for scale in ["linear", "log"]:

            galleryDetails = \
            {
                "P_TEXT": f"{self.areaType.capitalize()}s of {self.widerArea} + {scale} scale"
            }
            gallery = indexPage.addGallery(scale, galleryDetails)

            for figureName in ["overview", "cases", "admissions", "deaths"]:
                fileName = os.path.join(dirName, f"{scale}", figureName + ".png")

                figureDetails = \
                {
                    "ALT_TEXT": f"{figureName} ({scale} scale)",
                    "CAPTION_TEXT": f"Daily {figureName} in {self.widerArea} using a {scale} scale."
                }
                figure = gallery.addFigure(figureName, f"{scale}", fileName, figureDetails)
                figure.createThumb(suffix="-thumb")

            html = indexPage.saveHtml()


    def plotAreas(self, period="daily"):
        """Plot charts for each area"""

        self.plotSummary(period)
        self.plotCases(period)
        self.plotAdmissions(period)
        self.plotDeaths(period)
        self.saveIndexPages()

In [5]:
print("Loading data...")

population = ons_population.Population()
population.loadYears(limit=1)

nations = Areas(areaType="nation")
nationLookup = {v: k for k, v in common_core.nations.items()}
for nationName in phe_core.nationNames:
    nationCode = nationLookup[nationName]
    nations.addArea(nationName, areaCode=nationCode)
    nations.loadArea(nationName)

regions = Areas(areaType="region")
for regionCode in common_core.regions:
    regionName = common_core.regions[regionCode]
    regions.addArea(regionName, areaCode=regionCode)
    regions.loadArea(regionName)

nhsRegions = Areas(areaType="nhsregion")
for nhsRegionName in common_core.nhsRegionNames:
    nhsRegions.addArea(nhsRegionName)
    nhsRegions.loadArea(nhsRegionName)

# Transfer patient data from NHS regions to standard regions
for areaName in common_core.nhsRegionMappings:
    nhsAreaName = common_core.nhsRegionMappings[areaName]
    periods = nhsRegions.areas[nhsAreaName].data

    for period in periods:
        periodData = periods[period]
        for dataType in periodData:
            regions.areas[areaName].data[period][dataType] = periodData[dataType]

print("\nAll done!")

Loading data...

All done!


## Draw Charts

In [6]:
nations.plotAreas()

regions.plotAreas()